# Start elastic search server first

In [1]:
from elasticsearch import Elasticsearch 
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])


# Create local index

## Clean index

In [38]:
es.indices.delete(index='covid-document-index', ignore=[400, 404])
es.indices.delete(index='covid-section-index', ignore=[400, 404])

{'acknowledged': True}

## Create index

In [16]:
es.indices.create(index='covid-document-index', ignore=400)
es.indices.create(index='covid-section-index', ignore=400)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'covid-section-index'}

## Load covid document and sections

In [17]:
import pandas as pd
from tqdm.auto import tqdm

In [18]:
covid_questions = pd.read_csv('../data/website_questions_21032020_13h00.csv')['question'].tolist()

In [19]:
from os import listdir
from os.path import isfile, join
mypath = '../scrape/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) if '.json' in f and 'faq' not in f]

In [20]:
import json

In [35]:
for i in tqdm(onlyfiles):
    with open(mypath + i, 'r', encoding='utf-8') as f:
        t = json.load(f)
        doc = {
            'content': json.dumps(t),
            'file_name': i,
        }
        tmp = es.index('covid-document-index', doc, id=i)
        c = 1
        for sec in t:
            rec = {
                'section': sec,
                'content': t[sec],
                'file_name': i
            }
            tmp = es.index('covid-section-index', rec, id=i+'_section_'+str(c))
            c += 1
            break
        break

In [36]:
tmp

{'_index': 'covid-section-index',
 '_type': '_doc',
 '_id': '0_en.json_section_1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [33]:
es.cat.count('covid-section-index', params={"format": "json"})

[{'epoch': '1584902620', 'timestamp': '18:43:40', 'count': '0'}]

In [34]:
es.cat.count('covid-document-index', params={"format": "json"})

[{'epoch': '1584902621', 'timestamp': '18:43:41', 'count': '1'}]

# Search

## Covid section/document

In [31]:
from tqdm.auto import tqdm
from copy import deepcopy

In [32]:
topk = 3

In [37]:
covid_section_answers = {}
covid_document_answers = {}
for i in tqdm(english_covid_questions):
    print('Question: ', i, '\n')
    res= es.search({'query':{
            'match':{
                "content": i
            }
        }
    },index='covid-section-index')
    covid_section_answers[i] = deepcopy(res['hits']['hits'])
    for j in range(topk):
        print('# {} Section: '.format(j + 1), covid_section_answers[i][j]['_source'], '\n')
    res= es.search({'query':{
            'match':{
                "content": i
            }
        }
    },index='covid-document-index')
    covid_document_answers[i] = deepcopy(res['hits']['hits'])
    print('# 1 Document sections: ', [i for i in json.loads(covid_document_answers[i][0]['_source']['content'])])
    print('# 1 Document: ',
          covid_document_answers[i][0]['_source']['content'],
          '\n')
    print('-' *  50)

Question:  What is Dialogue's position on sick notes? 

# 1 Section:  {'section': 'What does stress mean?', 'content': ['Stress is a normal physiological response to an abnormal situation. As such, it is part and parcel of our lives. It enables our body to adapt to the multiplicity of positive and negative events that we experience, like a birth, marriage, loss of employment, etc. Stress comes and goes on its own, depending of what factors are involved. For example, if you feel stressed on the job but less so at home in the evening or on the weekend, we could deduce that the stressors are work-related.'], 'file_name': '3_en.json'} 

# 2 Section:  {'section': 'Health recommendations for everyone', 'content': ['Wash your hands often with soap under warm running water for at least 20\xa0seconds.Use an alcohol-based hand rub if soap and water are not available.Practice proper cough and sneeze etiquette: \tCover your mouth and nose with your arm to reduce the spread of germs.If you use a ti

# 1 Section:  {'section': 'What does stress mean?', 'content': ['Stress is a normal physiological response to an abnormal situation. As such, it is part and parcel of our lives. It enables our body to adapt to the multiplicity of positive and negative events that we experience, like a birth, marriage, loss of employment, etc. Stress comes and goes on its own, depending of what factors are involved. For example, if you feel stressed on the job but less so at home in the evening or on the weekend, we could deduce that the stressors are work-related.'], 'file_name': '3_en.json'} 

# 2 Section:  {'section': 'Measures taken by the government', 'content': ['The Gouvernement du Québec has taken a number of measures to contain the spread, including:', 'the adoption, on March 13, 2020, of an Order in Council declaring a health emergency throughout Québec\xa0territory\xa0(PDF\xa019.36 Kb)\xa0(in French only). This exceptional measure will, by way of an example, enable the Ministère de la Santé e

# 1 Document:  {"Health recommendations for everyone": ["Wash your hands often with soap under warm running water for at least 20\u00a0seconds.Use an alcohol-based hand rub if soap and water are not available.Practice proper cough and sneeze etiquette: \tCover your mouth and nose with your arm to reduce the spread of germs.If you use a tissue, dispose of it as soon as possible and wash your hands afterwards.If you are sick, avoid contact with more vulnerable people, including older adults and people who have a chronic disease. For example, avoid visiting people in hospital, people living in residential and long-term care centres or in private residences.Avoid direct contact when you greet someone, such as shaking hands, and use alternative ways to greet people.", "Every individual has their part to play to reduce the risk of spreading viruses.", "Masks are not an effective means of protection for the general public. Their use is indicated more for patients who are suspected of being in

# 3 Section:  {'section': 'Health recommendations for everyone', 'content': ['Wash your hands often with soap under warm running water for at least 20\xa0seconds.Use an alcohol-based hand rub if soap and water are not available.Practice proper cough and sneeze etiquette: \tCover your mouth and nose with your arm to reduce the spread of germs.If you use a tissue, dispose of it as soon as possible and wash your hands afterwards.If you are sick, avoid contact with more vulnerable people, including older adults and people who have a chronic disease. For example, avoid visiting people in hospital, people living in residential and long-term care centres or in private residences.Avoid direct contact when you greet someone, such as shaking hands, and use alternative ways to greet people.', 'Every individual has their part to play to reduce the risk of spreading viruses.', 'Masks are not an effective means of protection for the general public. Their use is indicated more for patients who are su

# 1 Document:  {"Health recommendations for everyone": ["Wash your hands often with soap under warm running water for at least 20\u00a0seconds.Use an alcohol-based hand rub if soap and water are not available.Practice proper cough and sneeze etiquette: \tCover your mouth and nose with your arm to reduce the spread of germs.If you use a tissue, dispose of it as soon as possible and wash your hands afterwards.If you are sick, avoid contact with more vulnerable people, including older adults and people who have a chronic disease. For example, avoid visiting people in hospital, people living in residential and long-term care centres or in private residences.Avoid direct contact when you greet someone, such as shaking hands, and use alternative ways to greet people.", "Every individual has their part to play to reduce the risk of spreading viruses.", "Masks are not an effective means of protection for the general public. Their use is indicated more for patients who are suspected of being in

...

# 1 Section:  {'section': 'About coronaviruses', 'content': ['In December\xa02019, the World Health Organization was alerted to several cases of pneumonia in Wuhan, China. The virus did not match any other known virus. On January 7, 2020, China confirmed COVID-19.', 'For more information, see the\xa0Situation of the coronavirus (COVID-19) in Québec\xa0page.', 'About coronaviruses', 'SARS‑CoV‑2 belongs to the coronavirus family. Some viruses in this family cause disease in animals, while others cause disease in humans. In rare cases, coronaviruses that infect animals can also infect people. In Québec, there is no evidence to suggest a risk of human infection caused by animals or animal products. Coronaviruses can be spread between people through close contact. In addition to\xa0SARS‑CoV‑2, two other coronaviruses are transmitted from animals to humans and have caused serious diseases in humans, namely, SARS‑CoV in\xa02003 and MERS‑CoV since\xa02012.'], 'file_name': '2_en.json'} 

# 2 Se

# 1 Document:  {"About coronaviruses": ["In December\u00a02019, the World Health Organization was alerted to several cases of pneumonia in Wuhan, China. The virus did not match any other known virus. On January 7, 2020, China confirmed COVID-19.", "For more information, see the\u00a0Situation of the coronavirus (COVID-19) in Qu\u00e9bec\u00a0page.", "About coronaviruses", "SARS\u2011CoV\u20112 belongs to the coronavirus family. Some viruses in this family cause disease in animals, while others cause disease in humans. In rare cases, coronaviruses that infect animals can also infect people. In Qu\u00e9bec, there is no evidence to suggest a risk of human infection caused by animals or animal products. Coronaviruses can be spread between people through close contact. In addition to\u00a0SARS\u2011CoV\u20112, two other coronaviruses are transmitted from animals to humans and have caused serious diseases in humans, namely, SARS\u2011CoV in\u00a02003 and MERS\u2011CoV since\u00a02012."], "Sy



--------------------------------------------------
Question:  How do I know when to get tested? 

# 1 Section:  {'section': 'Take care of yourself', 'content': ['Be attentive to your feelings, emotions and reactions and allow yourself to voice them to someone you trust. Write them down or express them through physical or other types of activity.Make use of physical activity to let the stress out and eliminate tension.Practice healthy living habits like proper nutrition and sufficient sleep.Limit your access to stressors.Allow yourself life’s little pleasures such as listening to music, taking a warm bath, reading, etc.Remain in contact with people that do you good.Remind yourself of winning strategies you used in the past to get through difficult times.Count on your own strengths.Set limits for yourself, such as refusing a task that is non-essential or that you do not want to do.Learn to delegate and let others help you (this might be asking your children to do the dishes).'], 'file_

# 1 Document:  {"About coronaviruses": ["In December\u00a02019, the World Health Organization was alerted to several cases of pneumonia in Wuhan, China. The virus did not match any other known virus. On January 7, 2020, China confirmed COVID-19.", "For more information, see the\u00a0Situation of the coronavirus (COVID-19) in Qu\u00e9bec\u00a0page.", "About coronaviruses", "SARS\u2011CoV\u20112 belongs to the coronavirus family. Some viruses in this family cause disease in animals, while others cause disease in humans. In rare cases, coronaviruses that infect animals can also infect people. In Qu\u00e9bec, there is no evidence to suggest a risk of human infection caused by animals or animal products. Coronaviruses can be spread between people through close contact. In addition to\u00a0SARS\u2011CoV\u20112, two other coronaviruses are transmitted from animals to humans and have caused serious diseases in humans, namely, SARS\u2011CoV in\u00a02003 and MERS\u2011CoV since\u00a02012."], "Sy



--------------------------------------------------
Question:  Must the fever-symptom related to COVID-19 be a continuous fever, or is it possible that the fever is short-lived? 

# 1 Section:  {'section': 'For citizens returning from travelling', 'content': ['Voluntary self-isolation\xa0\xa0for 14\xa0days is recommended for anyone returning from abroad on or after March\xa012,\xa02020. Returning travellers must be vigilant and monitor themselves for symptoms.', 'Businesses that deliver goods from abroad in Québec do not have to ask their employees to temporarily self-isolate.', 'Self-isolation\xa0\xa0is mandatory for all public service employees and health care, education and daycare workers, both private and public, returning from abroad. Public sector employees will continue their treatment during the isolation period.', 'Individuals who are returning from outside Canada and are in self-isolation at home must abide by the advice indicated below.', 'Stay at home', 'Do not go to scho